In [8]:
!pip install nltk tensorflow tflearn


     -------------------------------------- 107.3/107.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 266.3/266.3 MB 15.2 MB/s eta 0:00:00
     ---------------------------------------- 439.2/439.2 kB ? eta 0:00:00
     ------------------------------------- 895.9/895.9 kB 28.6 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 47.8 MB/s eta 0:00:00
     --------------------------------------- 23.2/23.2 MB 40.9 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 54.0 MB/s eta 0:00:00
     ---------------------------------------- 126.5/126.5 kB ? eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 47.6 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 63.6 MB/s eta 0:00:00
     ---------------------------

In [3]:
import nltk
import json

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Thanh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
intents =  {"intents": [
        {"tag": "greeting",
         "patterns": ["Hi there", "Is anyone there?","Hey","Hola", "Hello", "Good day"],
         "responses": ["Hello", "Good to see you again", "Hi there, how can I help?"],
         "context": [""]
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],
         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],
         "context": [""]
        },
        {"tag": "query_greeting",
         "patterns": ["How are you?", "How are you doing?", "What's up?", "How ya?", "How are you", "How are you doing", "Hows everything?", "How's everything", "What's new?"],
         "responses": ["I'm great, Thank you, How are you?", "I'm good, how are you?", "I'm doing fine, how are you?"],
         "context": [""]
        },
        {"tag": "res_greeting",
         "patterns": ["I'm great, Thank you for asking", "I'm Fine", "I'm doing fine", "I'm doing great", "Im fine", "im great"],
         "responses": ["That's great to hear! How can I help you?"],
         "context": [""]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me", "No thanks", "No thank you"],
         "responses": ["My pleasure", "You're Welcome"],
         "context": [""]
        },
        {"tag": "query",
         "patterns": ["I have a question", "I would like to ask you something", "I want to ask something", "Can you help me?", "I need help", "Can you help me?"],
         "responses": ["What may it be?", "How can I help you?", "What is it?"],
         "context": [""]
        },
        {"tag": "refer",
         "patterns": ["I want to know", "How can you do", "How can i do", "What to do when", "What to do if", "How can they do", "I would like to know", "I want to know if", "Is it possible to"],
         "responses": ["I don't know the answer for your question. What I do know is that you can easily google it"],
         "context": [""]
        }
    ]
 }

In [5]:
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import pickle
import random

convert_data = json.dumps(intents)
data = json.loads(convert_data)
stemmer = LancasterStemmer()

words = []
labels = []
x_docs = []
y_docs = []

# getting informations from intents.json--
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        x_docs.append(wrds)
        y_docs.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

# Stemming the words and removing duplicate elements.
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []
out_empty = [0 for _ in range(len(labels))]

# One hot encoding, Converting the words to numerals
for x, doc in enumerate(x_docs):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)


    output_row = out_empty[:]
    output_row[labels.index(y_docs[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

In [6]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 2999  | total loss: 0.16651 | time: 0.006s
| Adam | epoch: 500 | loss: 0.16651 - acc: 0.9890 -- iter: 40/48
Training Step: 3000  | total loss: 0.15054 | time: 0.007s
| Adam | epoch: 500 | loss: 0.15054 - acc: 0.9901 -- iter: 48/48
--
INFO:tensorflow:C:\Users\Thanh\Documents\Informatica\Real Fake Real Impact\Rob_AI\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for s_word in s_words:
        for i, w in enumerate(words):
            if w == s_word:
                bag[i] = 1

    return np.array(bag)

def chat():

    while True:
        inp = input("\n\nYou: ")
        if inp.lower() == 'quit':
            break

    #Porbability of correct response 
        results = model.predict([bag_of_words(inp, words)])

    # Picking the greatest number from probability
        results_index = np.argmax(results)

        tag = labels[results_index]

        for tg in data['intents']:

            if tg['tag'] == tag:
                responses = tg['responses']
                print("Bot:\t" + random.choice(responses))

In [ ]:
chat()



You: hello
Bot:	Good to see you again
Bot:	I'm sorry I didn't quite understand. Could you reprhase your sentence?
Bot:	I'm sorry I didn't quite understand. Could you reprhase your sentence?
Bot:	I'm sorry I didn't quite understand. Could you reprhase your sentence?
Bot:	I'm sorry I didn't quite understand. Could you reprhase your sentence?
Bot:	I'm sorry I didn't quite understand. Could you reprhase your sentence?
Bot:	I'm sorry I didn't quite understand. Could you reprhase your sentence?
